# Forecasting Optuna Search CV 
## Modulus Set 2

**Notebook Goal**
- A modeling pipeline that optimizes the hyperparameters of the sktime forecasters that have the [capavility:pred_int tag](https://www.sktime.net/en/stable/examples/01b_forecasting_proba.html) 
- This notebook will focus on the ones where `i mod 4 = 2` wher `i` is the index of the registry table in the above link.
- The work will be based on this documentation: [ForecastingOptunaSearchCV](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingOptunaSearchCV.html)

## Data Load

In [1]:
from sktime.forecasting.model_selection import (
    ForecastingOptunaSearchCV,
    )
from sktime.datasets import load_shampoo_sales
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from sktime.forecasting.base import ForecastingHorizon
from sktime.split import ExpandingWindowSplitter
from sktime.split import temporal_train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import STLForecaster
import optuna
from  optuna.distributions import CategoricalDistribution

/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
y = load_shampoo_sales()
y_train, y_test = temporal_train_test_split(y=y, test_size=6)
fh = ForecastingHorizon(y_test.index, is_relative=False).to_relative(
cutoff=y_train.index[-1]
    )
cv = ExpandingWindowSplitter(fh=fh, initial_window=24, step_length=1)
forecaster = TransformedTargetForecaster(
    steps=[
        ("detrender", Detrender()),
        ("deseasonalizer", Deseasonalizer()),
        ("scaler", TabularToSeriesAdaptor(RobustScaler())),
        ("minmax2", TabularToSeriesAdaptor(MinMaxScaler((1, 10)))),
        ("forecaster", NaiveForecaster()),
        ]
    )
param_grid = {
    "scaler__transformer__with_scaling": CategoricalDistribution((True, False)),
    "forecaster": CategoricalDistribution((STLForecaster(), ThetaForecaster())),
}
gscv = ForecastingOptunaSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv,
    n_evals=10,
    )
gscv.fit(y)
print(f"{gscv.best_params_=}")

/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains STLForecaster() which is of type STLForecaster.
  warnings.warn(message)
/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains ThetaForecaster() which is of type ThetaForecaster.
  warnings.warn(message)
/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/sktime/forecasting/model_selection/_tune.py:1773: UserWarning: ForecastingOptunaSearchCV is experimental, and interfaces may change. User feedback and suggestions for future development ar

gscv.best_params_={'scaler__transformer__with_scaling': False, 'forecaster': STLForecaster()}


In [7]:
y

Period
1991-01    266.0
1991-02    145.9
1991-03    183.1
1991-04    119.3
1991-05    180.3
1991-06    168.5
1991-07    231.8
1991-08    224.5
1991-09    192.8
1991-10    122.9
1991-11    336.5
1991-12    185.9
1992-01    194.3
1992-02    149.5
1992-03    210.1
1992-04    273.3
1992-05    191.4
1992-06    287.0
1992-07    226.0
1992-08    303.6
1992-09    289.9
1992-10    421.6
1992-11    264.5
1992-12    342.3
1993-01    339.7
1993-02    440.4
1993-03    315.9
1993-04    439.3
1993-05    401.3
1993-06    437.4
1993-07    575.5
1993-08    407.6
1993-09    682.0
1993-10    475.3
1993-11    581.3
1993-12    646.9
Freq: M, Name: Number of shampoo sales, dtype: float64

In [6]:
gscv


ForecastingOptunaSearchCV(cv=ExpandingWindowSplitter(fh=ForecastingHorizon([1, 2, 3, 4, 5, 6], dtype='int64', is_relative=True),
                                                     initial_window=24),
                          forecaster=TransformedTargetForecaster(steps=[('detrender',
                                                                         Detrender()),
                                                                        ('deseasonalizer',
                                                                         Deseasonalizer()),
                                                                        ('scaler',
                                                                         TabularToSeriesAdaptor(transformer=RobustScaler())),
                                                                        ('minmax2',
                                                                         TabularToSeriesAdaptor(transformer=MinMaxScaler(feature_range=(1, 10)))),
                                                                        ('forecaster',
                                                                         NaiveForecaster())]),
                          n_evals=10,
                          param_grid={'forecaster': CategoricalDistribution(choices=(STLForecaster(), ThetaForecaster())),
                                      'scaler__transformer__with_scaling': CategoricalDistribution(choices=(True, False))})